In [1]:
import copy
import random
import numpy

white = 'w'
black = 'b'

In [2]:
def MoveError(Exception):
    pass

In [3]:
class Roll():
    def __init__(self, d=None):
        if d is None:
            self.roll = [random.choice(range(1, 7)) for i in range(2)]
        else:
            assert isinstance(d,list), 'Given roll must be parsed as a list.'
            if max(d)>6 or min(d)<1:
                raise Exception('Invalid roll.')
            else:
                self.roll = d
        if self.roll[0] == self.roll[1]:
            self.touse = [self.roll[0] for i in range(4)]
        else:
            self.touse = self.roll[:]
    def use(self, x):
        self.touse.remove(x)
    def __repr__(self):
        return str(self.touse)

In [20]:
class Turn():
    def __init__(self, player, roll):
        self.player = player
        self.roll = roll
        
    def is_complete(self):
        if len(self.roll.touse) == 0:
            return True
        else:
            return False
    def copy(self):
        return copy.deepcopy(self)
    def __repr__(self):
        return "Player: {}, Roll: {}".format(str(self.player), str(self.roll))

In [220]:
class Board():
    def __init__(self, spec=None):
        self.board = dict(zip(range(24),[None for i in range(24)]))
        self.prison = {}
        self.prison[white] = 0
        self.prison[black] = 0
        self.home = {}
        self.home[white] = 0
        self.home[black] = 0
        self.state = {}
        self.state[white] = None
        self.state[black] = None
        self.value = 1
        self.initialise_board(spec)
    
    def eval_state(self):
        self.state[white]= 0
        self.state[black] = 0
        for k,v in self.board.items():
            if v and v[0] == white:
                self.state[white] += (k+1)*v[1]      
            elif v and v[0] == black:
                self.state[black] += (24-k)*v[1]
        
    def initialise_board(self, spec):
        if spec is None:
            self.board[5] = [white, 5]
            self.board[7] = [white, 3]
            self.board[12] = [white, 5]
            self.board[23] = [white, 2]
            self.board[0] = [black, 2]
            self.board[11] = [black, 5]
            self.board[16] = [black, 3]
            self.board[18] = [black, 5]
        else:
            for k,v in spec.items():
                self.board[k] = v
        self.eval_state()
    
    def is_complete(self):
        if self.home[white] == 15:
            return True, 'White'
        elif self.home[black] == 15:
            return True, 'Black'
        else:
            return False, None
        
    def color_pieces(self, col):
        return [k for k,v in self.board.items() if v and v[0]==col]
    
    def furthest_from_home(self, col):
        if col == white:
            return max([k for k,v in self.board.items() if v and v[0]==white])
        else:
            return min([k for k,v in self.board.items() if v and v[0]==black])
        
    def copy(self):
        new = copy.deepcopy(self)
        return new
    
    def double(self):
        self.double*=2
        
    def pop(self, i):
        if self.board[i][1] > 1:
            self.board[i][1]-=1
        elif self.board[i][1] == 1:
            self.board[i] = None
        else:
            raise Exception('No points to remove.')
            
    def possible_moves(self, turn):
        col = turn.player
        roll = turn.roll.touse
        if len(roll) == 0:
            return []
        poss_moves = {}
        for d in roll:
            poss_moves[d] = []
        if self.prison[col]>0:
            if col == white:
                for d in roll:
                    if self.board[24-d] is None or self.board[24-d][1]<=1 or self.board[24-d][0] == white:
                        poss_moves[d].append(['p', 24-d])
            else:
                for d in roll:
                    if self.board[d-1] is None or self.board[d-1][1]<=1 or self.board[d-1][0] == black:
                        poss_moves[d].append(['p', d-1])
            return poss_moves
        f = self.furthest_from_home(col)
        for d in roll:
            if (col==white and f<=5):
                if f <= d-1:
                    poss_moves[d].append([f, 'h'])
                elif self.board[d-1] and self.board[d-1][0]==col and self.board[d-1][1]>0:
                    poss_moves[d].append([d-1, 'h'])
            elif (col==black and f>=18):
                if f >= 24-d:
                    poss_moves[d].append([f, 'h'])
                elif self.board[24-d] and self.board[24-d][0]==col and self.board[24-d][1]>0:
                    poss_moves[d].append([24-d, 'h'])
            
            pieces = self.color_pieces(col)
            for p in pieces:
                if col == white:
                    if p-d in pieces or (p-d>=0 and (self.board[p-d] is None or self.board[p-d][1]<=1)):
                            poss_moves[d].append([p, p-d])
                if col == black:
                    if p+d in pieces or (p+d<=23 and (self.board[p+d] is None or self.board[p+d][1]<=1)):
                        poss_moves[d].append([p, p+d])
        return poss_moves 
    
    @classmethod
    def is_in(cls, possiblemoves, move):
        for k, v in possiblemoves.items():
            if move in v:
                return k
        return False
    
    def move(self, col, src, dst):
        if self.prison[col]>0 and src != 'p':
            raise Exception('Must empty the prison.')
        if src == 'p':
            if self.board[dst] == None or self.board[dst][0] == col:
                new = self.copy()
                new.prison[col]-=1
                if new.board[dst]:
                    new.board[dst][1]+=1
                else:
                    new.board[dst] = [col, 1]
                new.eval_state()
                return new
            elif self.board[dst][0] != col and self.board[dst][1] == 1:
                new = self.copy()
                new.prison[new.board[dst][0]] += 1
                new.board[dst] = [col, 1]
                new.eval_state()
                return new
            else:
                raise Exception('Cannot free piece at this position.')
        elif dst == 'h':
            if self.prison[col]>0:
                raise Exception('Must empty the prison first.')
            elif (col==white and self.furthest_from_home(col)<=5) or col==black and self.furthest_from_home(col) >=18:
                    new = self.copy()
                    new.home[col]+=1
                    new.pop(src)
                    new.eval_state()
                    return new
            else:
                raise Exception('All pieces are not yet in the safe zone.')
        else:
            if self.board[src] is None or src not in self.color_pieces(col):
                raise Exception('No {} piece to move at {}'.format(col, src))
            elif dst<0 or dst>=24:
                raise Exception('Cannot move outside of the board.')
            elif self.board[dst] and self.board[dst][0] != col and self.board[dst][1] >1:
                raise Exception('Cannot move to this position, protected by opponent')
            else:
                new = self.copy()
                if self.board[dst] is None:
                    new.board[dst] = [col, 1]
                elif self.board[dst][0] != col and self.board[dst][1] == 1:
                    new.prison[self.board[dst][0]] += 1
                    new.board[dst] = [col, 1]
                else:
                    new.board[dst][1] += 1
                new.pop(src)
                new.eval_state()
                return new
            
    def __repr__(self):
        upp = [[' ' for i in range(12)] for j in range(5)]
        low = [[' ' for i in range(12)] for j in range(5)]
        for k in range(12):
            if self.board[k] is None:
                continue
            else:
                if self.board[k][0] == white:
                    fill = 'o'
                else:
                    fill = 'x'
                if self.board[k][1] <= 5:
                    for i in range(self.board[k][1]):
                        low[i][11-k] = fill
                else:
                    low[0][11-k] = str(self.board[k][1])
                    for i in range(1, 5):
                        low[i][11-k] = fill
                        
        for k in range(12, 24):
            if self.board[k] is None:
                continue
            else:
                if self.board[k][0] == white:
                    fill = 'o'
                else:
                    fill = 'x'
                if self.board[k][1] <= 5:
                    for i in range(self.board[k][1]):
                        upp[i][k-12] = fill
                else:
                    upp[0][k-12] = str(self.board[k][1])
                    for i in range(1, 5):
                        upp[i][k-12] = fill
        low_str = '======================================\n'
        for j in range(4,-1,-1):
            low_str += '|'+'  '.join(low[j][:6])+' | '+'  '.join(low[j][6:])+' |\n'
        low_str += '======================================\n 11 10 9  8  7  6   5  4  3  2  1  0'
        
        upp_str = ' 12 13 14 15 16 17 18 19 20 21 22 23\n======================================\n'
        for j in range(5):
            upp_str += '|'+'  '.join(upp[j][:6])+' | '+'  '.join(upp[j][6:])+' |\n'
        upp_str += '======================================\n'
        
        prison = ['|', 'o']+[' ' for s in range(34)]+['x', '|', '\n']
        prison[15:21] = 'PRISON'
        prison[3] = str(self.prison[white])
        prison[34] = str(self.prison[black])
        prison_str = ''.join(prison)
        
        
        if self.home[white] > 0:
            home_white_str = '\n             EATEN  {}  \n'.format(self.home[white])
            if self.home[black] > 0:
                home_black_str = '            EATEN  {}  \n'.format(self.home[black])
                return home_black_str+upp_str+prison_str+low_str+home_white_str
            else:
                 return upp_str+prison_str+low_str+home_white_str
        else:
            if self.home[black] > 0:
                home_black_str = '    EATEN  {}  \n'.format(self.home[black])
                return home_black_str+upp_str+prison_str+low_str
            else:
                 return upp_str+prison_str+low_str

In [221]:
class Game():
    def __init__(self, board=None, turn=None):
        if board:
            self.board = board
        else:
            self.board = Board()
        if turn:
            self.history = [turn]
        else:
            self.history = []
            
    def initiate_game(self, r0=None):
        if r0:
            pass
        else:
            r0 = Roll()
        if r0.roll[0]>r0.roll[1]:
            return Turn(white, r0)
        elif r0.roll[1]>r0.roll[0]:
            return Turn(black, r0)
        else:
            self.board.value *= 2
            return self.initiate_game()
            
    @classmethod
    def interact(cls, board, turn):
        temp_board = board.copy()
        temp_turn = turn.copy()
        if turn.player == white:
            print "White Player's Turn // Dices to play: {}".format(turn.roll)
            new_player = black
        else:
            print "Black Player's Turn // Dices to play: {}".format(turn.roll)
            new_player = white
            
        while not turn.is_complete():
            print temp_board
            print
            nextmove = raw_input('{} PLAYER: what is your next move? (You can still use these dice: {}) | '.format(temp_turn.player.upper(), temp_turn.roll))
            if nextmove.lower() == 'double':
                temp_board.double()
                continue
            elif 'show' in nextmove.lower():
                print 'Potential Moves: '
                possiblemoves = temp_board.possible_moves(temp_turn)
                if len(possiblemoves) == 0:
                    print 'NO VALID MOVES TO PLAY, MUST FINALIZE'
                else:
                    print possiblemoves
                continue
            elif nextmove.lower() == 'reset':
                temp_board = board.copy()
                temp_turn = turn.copy()
                continue
            elif nextmove.lower() == 'finalize':
                board = temp_board
                turn = temp_turn
                break
            elif nextmove.lower() in ['quit', 'exit']:
                return 'exit', 'exit'
            nextmove = [int(x) if x.isdigit() else x for x in nextmove.split()]
            possiblemoves = temp_board.possible_moves(temp_turn)
            if all(l == 0 for l in [len(v) for v in possiblemoves.values()]):
                print 'NO VALID MOVES, MUST FINALIZE'
                break
            which = Board.is_in(possiblemoves, nextmove)
            if  which == False:
                print "Invalid Move."
                continue
            else:
                temp_board = temp_board.move(temp_turn.player, *nextmove)
                temp_turn.roll.use(which)
        return board, Turn(new_player, Roll())
    
    def play(self, pickup=False):
        if pickup == False:
            turn = self.initiate_game(Roll())
        else:
            turn = self.history[0]
        print turn
        self.history.append(turn)
        while not self.board.is_complete()[0]:
            self.board, turn = Game.interact(self.board, turn)
            self.history.append(turn)
            if turn == 'exit':
                break
        try:
            print 'GAME CONCLUDED, {} wins !'.format(self.board.is_complete()[1])
        except:
            print 'GAME ABORTED'

In [189]:
g = Game()
g.play()

Player: w, Roll: [6, 3]
White Player's Turn // Dices to play: [6, 3]
 12 13 14 15 16 17 18 19 20 21 22 23
|o           x    | x              o |
|o           x    | x              o |
|o           x    | x                |
|o                | x                |
|o                | x                |
|o 0           PRISON             0 x|
|x                | o                |
|x                | o                |
|x           o    | o                |
|x           o    | o              x |
|x           o    | o              x |
 11 10 9  8  7  6   5  4  3  2  1  0

W PLAYER: what is your next move? (You can still use these dice: [6, 3]) | 23 17
 12 13 14 15 16 17 18 19 20 21 22 23
|o           x  o | x              o |
|o           x    | x                |
|o           x    | x                |
|o                | x                |
|o                | x                |
|o 0           PRISON             0 x|
|x                | o                |
|x                | o               

In [215]:
s = {}
s[22] = [black, 2]
s[21] = [black, 2]
s[20] = [black, 2]
s[19] = [black, 2]
s[18] = [black, 4]
s[17] = [black, 1]
s[16] = [black, 2]
s[1] = [white, 2]
s[2] = [white, 2]
s[3] = [white, 2]
s[4] = [white, 2]
s[5] = [white, 3]
s[6] = [white, 2]
s[8] = [white, 1]
saved = Board(s)
saved.prison[white] = 1
saved

 12 13 14 15 16 17 18 19 20 21 22 23
|            x  x | x  x  x  x  x    |
|            x    | x  x  x  x  x    |
|                 | x                |
|                 | x                |
|                 |                  |
|o 1           PRISON             0 x|
|                 |                  |
|                 |                  |
|                 | o                |
|               o | o  o  o  o  o    |
|         o     o | o  o  o  o  o    |
 11 10 9  8  7  6   5  4  3  2  1  0

In [216]:
g = Game(saved, Turn(player=white, roll=Roll([5,4])))
g.play(True)

Player: w, Roll: [5, 4]
White Player's Turn // Dices to play: [5, 4]
 12 13 14 15 16 17 18 19 20 21 22 23
|            x  x | x  x  x  x  x    |
|            x    | x  x  x  x  x    |
|                 | x                |
|                 | x                |
|                 |                  |
|o 1           PRISON             0 x|
|                 |                  |
|                 |                  |
|                 | o                |
|               o | o  o  o  o  o    |
|         o     o | o  o  o  o  o    |
 11 10 9  8  7  6   5  4  3  2  1  0

W PLAYER: what is your next move? (You can still use these dice: [5, 4]) | finalize
Black Player's Turn // Dices to play: [1, 5]
 12 13 14 15 16 17 18 19 20 21 22 23
|            x  x | x  x  x  x  x    |
|            x    | x  x  x  x  x    |
|                 | x                |
|                 | x                |
|                 |                  |
|o 1           PRISON             0 x|
|                 |        

AttributeError: 'list' object has no attribute 'values'